In [1]:
# from huggingface_hub import snapshot_download

repo_id = 'allenai/pixmo-cap'
# target_dir = '/lambdafs/jacob/data/' + repo_id.replace('/', '_')

from datasets import load_dataset
dataset = load_dataset(repo_id, split='train')

# snapshot_download(repo_id=repo_id, local_dir=target_dir, repo_type='dataset')


In [2]:
%load_ext autoreload
%autoreload 2

# import nest_asyncio
# nest_asyncio.apply()

In [3]:
from data_manager import DataManager

In [4]:
manager = DataManager(db_path='/home/jacob/Desktop/test_db.db')

In [ ]:
from utils.download_manager import download_manager
import random
task_queue, result_queue, stop_event, workers = download_manager(num_workers=30)

In [ ]:
from datasets import load_dataset
import json

def process_batch(batch):
    return {
        'data': [
            json.dumps({"system_prompt": system_prompt, "question": question, "response": response})
            for system_prompt, question, response in zip(
                batch['system_prompt'], batch['question'], batch['response']
            )
        ]
    }

def CONVERT(download_dir):
    dataset = load_dataset("Open-Orca/OpenOrca", split='train')
    processed_dataset = dataset.map(process_batch, batched=True, batch_size=10000)
    
    for example in processed_dataset:
        yield example

# # Iterate through the processed dataset
# processed_dataset = CONVERT("/home/jacob/Desktop/test")
# for example in processed_dataset:
#     print(example)
#     break

# manager.insert_from_converter(CONVERT, "/home/jacob/Desktop/test", data_source='Open-Orca/OpenOrca')

In [6]:
import random
def batch_process(examples):
    """
    Process multiple examples at once in a batch.
    
    Args:
        examples: Dictionary of lists where each key corresponds to a column
                 and each list contains the values for that column
    
    Returns:
        Dictionary with processed data
    """
    urls = examples["image_url"]
    downloads = [{"url": url, "path": "/home/jacob/Desktop/data/pixmo/"+str(random.randint(100000, 999999))+url.split("/")[-1].split("?")[0][-100:]} for url in urls]
    for dl in downloads:
        task_queue.put(dl)
    data = {"data": examples["caption"], "files": [str([dl["path"]]) for dl in downloads]}
    manager.insert_data(data, data_source='Open-Orca/OpenOrca', group_id="testing", data_type="language")
    return data

# Apply the batch processing function to the dataset
dataset_clean = dataset.map(
    batch_process,
    batched=True,
    batch_size=1000
)

Parameter 'function'=<function batch_process at 0x1555297f2200> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/717042 [00:00<?, ? examples/s]

In [73]:
data = manager.sample_data(data_sources=['Open-Orca/OpenOrca'], group_ids=["testing", "testing3"], limit=100, offset=10)
data[0]

{'uuid': 'ab4153a1-5ff8-4c30-b2bd-b6c65a67074e',
 'data_source': 'Open-Orca/OpenOrca',
 'group_id': 'testing',
 'data_type': 'language',
 'data': '{"system_prompt": "You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer.", "question": "Q: Answer the following question given this paragraph:   The kidneys also secrete hormones that help maintain homeostasis. For example, they produce a hormone that stimulates bone marrow to produce red blood cells when more are needed. They also secrete a hormone that regulates blood pressure and keeps it in a normal range.   Q: What organs secrete hormones that help maintain homeostasis?   A:\nThe answer is:", "response": "The kidneys are the organs that secrete hormones to help maintain homeostasis. They produce a hormone that stimulates bone marrow to produce red blood cells when needed, and they also secrete a hormone that regulates blood pressure, keeping it within a normal range."}',
 'file

In [36]:
manager.delete_group(group_id="testing")

0

In [22]:
test_manager = DownloadManager(target_dir)
test_manager.download(["ShareGPTo"])